# 2. Managing System dependencies
http://wiki.ros.org/ROS/Tutorials/rosdep

## setup environment

In [1]:
import os
import sys 

home_dir = os.getcwd()
home_dir

'/home/pi/notebooks/Tutorials/1_Core_ROS_Tutorials/Intermediate_Level'

In [2]:
# get ROS related environment variables
envs = !printenv | grep ROS 
ROS_ENVs = {}

for env in envs:
    env = env.split('=')
    [k, v] = env
    ROS_ENVs[k] = v

ROS_ENVs['LD_LIBRARY_PATH'] = '/opt/ros/kinetic/lib'
ROS_ENVs['PKG_CONFIG_PATH'] = '/opt/ros/kinetic/lib/pkgconfig'
ROS_ENVs['CMAKE_PREFIX_PATH'] = '/opt/ros/kinetic'  
del ROS_ENVs['PWD']

In [3]:
# define workspace path
ws_dir = 'catkin_ws'
ws_dir_abs = os.path.join(home_dir, ws_dir)
src_dir_abs = os.path.join(ws_dir_abs, 'src') 

# append environment paths
def gen_paths(ws_dir_abs, src_dir_abs):
    paths = {}
    paths['ROS_PACKAGE_PATH'] = src_dir_abs
    paths['LD_LIBRARY_PATH'] = os.path.join(ws_dir_abs + 'devel/lib')
    paths['ROSLISP_PACKAGE_DIRECTORIES'] = os.path.join(ws_dir_abs + 'devel/share/common-lisp')
    paths['PKG_CONFIG_PATH'] = os.path.join(ws_dir_abs + 'devel/lib/pkgconfig')
    paths['CMAKE_PREFIX_PATH'] = os.path.join(ws_dir_abs + 'devel/lib')
    paths['LD_LIBRARY_PATH'] = os.path.join(ws_dir_abs + 'devel')
    return paths

def append_paths(ROS_ENVs, paths): 
    for path in ROS_ENVs.keys():
        os.environ[path] = ROS_ENVs[path]
        if path in paths.keys():
            os.environ[path] = '{}:{}'.format(paths[path], os.environ[path])

append_paths(ROS_ENVs, 
             gen_paths(ws_dir_abs, src_dir_abs))

## System Dependencies
ROS packages sometimes require external libraries and tools that must be provided by the operating system. These required libraries and tools are commonly referred to as system dependencies. In some cases these system dependencies are not installed by default. ROS provides a simple tool, ***rosdep***, that is used to download and install system dependencies.

ROS packages must declare that they need these system dependencies in the package manifest. Let's look at the manifest for the turtlesim package:

In [4]:
path = !rospack find rospy
path = path[0]

In [5]:
cd {path}

/opt/ros/kinetic/share/rospy


In [6]:
cat package.xml

<package>
  <name>rospy</name>
  <version>1.12.12</version>
  <description>
    rospy is a pure Python client library for ROS. The rospy client
    API enables Python programmers to quickly interface with ROS <a
    href="http://ros.org/wiki/Topics">Topics</a>, <a
    href="http://ros.org/wiki/Services">Services</a>, and <a
    href="http://ros.org/wiki/Parameter Server">Parameters</a>. The
    design of rospy favors implementation speed (i.e. developer
    time) over runtime performance so that algorithms can be quickly
    prototyped and tested within ROS. It is also ideal for
    non-critical-path code, such as configuration and initialization
    code. Many of the ROS tools are written in rospy to take
    advantage of the type introspection capabilities.

    Many of the ROS tools, such
    as <a href="http://ros.org/wiki/rostopic">rostopic</a>
    and <a href="http://ros.org/wiki/rosservice">rosservice</a>, are
    built on top of rospy.
  </description>
  <maintainer email="dtho

In [19]:
!rospack depends1 rospy

genpy
roscpp
rosgraph
rosgraph_msgs
roslib
std_msgs


### rosdep
rosdep is a tool you can use to install system dependencies required by ROS packages.

Usage:
```
rosdep install [package]
```

In [7]:
!rosdep install turtlesim

#All required rosdeps installed successfully


If you've been following along with the tutorials, it's likely that this is the first time you've used rosdep. When you run this command, you'll get an error message:
```
ERROR: your rosdep installation has not been initialized yet.  Please run:

    sudo rosdep init
    rosdep update
```
Just run those two commands and then try to install turtlesim's dependencies again.

If you installed using binaries you will see: 
```
All required rosdeps installed successfully
```
Otherwise you will see the output of installing the dependencies of turtlesim:
```
#!/usr/bin/bash

set -o errexit
set -o verbose


if [ ! -f /opt/ros/lib/libboost_date_time-gcc42-mt*-1_37.a ] ; then
  mkdir -p ~/ros/ros-deps
  cd ~/ros/ros-deps
  wget --tries=10 http://pr.willowgarage.com/downloads/boost_1_37_0.tar.gz
  tar xzf boost_1_37_0.tar.gz
  cd boost_1_37_0
  ./configure --prefix=/opt/ros
  make
  sudo make install
fi


if [ ! -f /opt/ros/lib/liblog4cxx.so.10 ] ; then
  mkdir -p ~/ros/ros-deps
  cd ~/ros/ros-deps
  wget --tries=10 http://pr.willowgarage.com/downloads/apache-log4cxx-0.10.0-wg_patched.tar.gz
  tar xzf apache-log4cxx-0.10.0-wg_patched.tar.gz
  cd apache-log4cxx-0.10.0
  ./configure --prefix=/opt/ros
  make
  sudo make install
fi
```

rosdep runs the bash script above and exits when complete.

### rosdistro/rosdep
While rosdep is the client tool, the reference is provided by rosdep rules, stored online in ros/rosdistro/rosdep on github.

When doing




In [9]:
!rosdep update

reading in sources list data from /etc/ros/rosdep/sources.list.d
Hit https://raw.githubusercontent.com/ros/rosdistro/master/rosdep/osx-homebrew.yaml
Hit https://raw.githubusercontent.com/ros/rosdistro/master/rosdep/base.yaml
Hit https://raw.githubusercontent.com/ros/rosdistro/master/rosdep/python.yaml
Hit https://raw.githubusercontent.com/ros/rosdistro/master/rosdep/ruby.yaml
Hit https://raw.githubusercontent.com/ros/rosdistro/master/releases/fuerte.yaml
Query rosdistro index https://raw.githubusercontent.com/ros/rosdistro/master/index.yaml
Add distro "groovy"
Add distro "hydro"
Add distro "indigo"
Add distro "jade"
Add distro "kinetic"
Add distro "lunar"
Add distro "melodic"
updated cache in /home/pi/.ros/rosdep/sources.cache


rosdep actually retrieves the rules from the rosdistro github repository.

These rules are used when a dependency is listed that doesn't match the name of a ROS package built on the buildfarm. Then rosdep checks if there exists a rule to resolve it for the proper platform and package manager you are using.

When creating a new package, you might need to declare new system dependencies to the rosdep rules if there are not there yet. Just edit the file, add the dependency needed (following a strict alphabetical order and a similar structure as the other dependencies already registered) and send a pull request.

After that pull request has been merged, you need to run :

In [10]:
!rosdep update

reading in sources list data from /etc/ros/rosdep/sources.list.d
Hit https://raw.githubusercontent.com/ros/rosdistro/master/rosdep/osx-homebrew.yaml
Hit https://raw.githubusercontent.com/ros/rosdistro/master/rosdep/base.yaml
Hit https://raw.githubusercontent.com/ros/rosdistro/master/rosdep/python.yaml
Hit https://raw.githubusercontent.com/ros/rosdistro/master/rosdep/ruby.yaml
Hit https://raw.githubusercontent.com/ros/rosdistro/master/releases/fuerte.yaml
Query rosdistro index https://raw.githubusercontent.com/ros/rosdistro/master/index.yaml
Add distro "groovy"
Add distro "hydro"
Add distro "indigo"
Add distro "jade"
Add distro "kinetic"
Add distro "lunar"
Add distro "melodic"
updated cache in /home/pi/.ros/rosdep/sources.cache


and now that dependency will be resolved by rosdep.

You can test it with :

In [11]:
!rosdep resolve rospy

#apt
ros-kinetic-rospy


where the first line is the package manager chosen for installing this dependency, and the second line is the actual name for that dependency on your current platform.